In [1]:
#%matplotlib notebook  
from matplotlib import pyplot as plt
import numpy as np
import pasttrec_ctrl as ptc
import json
import sigproc_kit
import tdc_daq as td
import baseline_calib
import ROOT

from my_utils import *
set_width_max_of_jupyterpad()

def fwhm(x,y):
  dummy, t1, tot = sigproc_kit.discriminate(x,y,np.max(y)/2.,0,0)
  return tot

def hist_rms(x,y):
  counts = np.sum(y)
  weights = y / counts
  mean = np.dot(x,weights)
  deviations = x - mean
  return np.sqrt(  np.dot(deviations**2,weights)   )
def list_rms(x):
  mean = sum(x)/len(x)
  deviations = x - mean
  return np.sqrt(  sum(deviations**2) /len(x)  )
def list_mean(x):
  mean = sum(x)/len(x)
  return mean

Welcome to JupyROOT 6.14/04


In [2]:
import db
print(db.board_list())

['0014', '0021', '0018', '0022', '0003', '0008', '0027', '0026', '0025', '0004', '0110', '0111', '0201', '0202', '0203', '0204', '0029', '0023', '0030', '0015', '0017', '0010', '0016', '0020', '0006', '0011', '0009', '0007', '0013', '0106', '0002', '0101', '0028', '0104', '0102', '0103', '0108', '0105', '0005']


In [3]:
#used_boards = ["0011"]
# used_boards = ["0016"]
used_boards = ["0201"]
#used_boards = db.board_list_installed()
#used_boards = used_boards[0:10]
#used_boards = ["0201","0202","0203","0204","0016","0011","0020"]
print(used_boards)
nscans = 1

## scan threshold of all boar
from functools import (reduce)
from math import (sqrt)
import numpy as np
import time
import tdc_daq as td
import matplotlib.pyplot as plt
import db



# rootMeanSquare :: [Num] -> Float
def rootMeanSquare(xs):
    return sqrt(reduce(lambda a, x: a + x * x, xs, 0) / len(xs))
def MeanArrays(xs,ys):
    return np.dot(xs,ys)/sum(ys)

#ptime_list = [ 10, 15, 20 ]
ptime_list = [ 20 ]
# pt_gain_list = [ 1, 2, 4 ]
pt_gain_list = [1]  

# for single board noise measurement, set all other boards to standby
for my_board in db.board_list():
  db.set_standby_board(my_board)



n_scan_steps = len(ptime_list)*len(pt_gain_list)*nscans

plt.rcParams["figure.figsize"] = (8,5)
trendtime_list = []
rmsAverageTrend_list = []
rmsAverageTrendError_list = []
meanAverageTrend_list = []
meanAverageTrendError_list = []

mean_scans = [[[] for b in range(len(used_boards)) ]  for i in range(n_scan_steps) ]
rms_scans =  [[[] for b in range(len(used_boards)) ]  for i in range(n_scan_steps) ]

scaler_list_channel = [[[] for i in range(n_scan_steps) ]  for i in range(16)] 
   




for ptime in ptime_list: 
 for pt_gain in pt_gain_list: 
  b = -1    
  for name in used_boards: 
   b += 1        
   scan_step = -1
   for scan in range(0,nscans): 
    scan_step += 1 
    print(scan_step,"board =" , name, "------------> peaking time = ", ptime, ", gain = ", pt_gain)
    #alert("next measure")
#     input("Press enter to continue")

    db.unset_standby_board(name)
    td.enable_tdc_channels_of_active_boards()
    ptc.init_active_boards()
    # pause for not killing trb by sending to many parallel commands regiser writes:
    time.sleep(5) 
    trendtime_list += [ name ]
    ptc.init_boards_by_name(used_boards, ptime, pt_gain)   
    print(len(trendtime_list),". scan ")
    #### baseline scan, all channels in parallel (same baseline set to all channels at once):
    if scan == 1:
            time.sleep(5) 
            print(" parallel baseline scan, all channels same baseline set at a step")        
            baseline_calib.baseline_calib_by_noise(name, dummy_calib=True) 
    #### baseline scan each channel individual, while other channels are "inactive", = baseline is minimal -15 or (maximal +15)
    if scan == 0:
           time.sleep(5) 
           print(" individual baseline scan, inactive = -15 ")
           baseline_calib.baseline_calib_by_noise(name, dummy_calib=True,individual=True) 
    if scan == 2:
       time.sleep(5) 
       print(" individual baseline scan, all other channesl active  baseline = calibrated individual baseline")
       bl_values = [0]*16
        # get the baselines calibrated for all 16 channels from database:
       calib = db.get_calib_json_by_name(name)
       if ("baselines" in calib):
           board_baselines = calib["baselines"]
           bl_values   = board_baselines[0:17]   
       print(bl_values)
       baseline_calib.baseline_calib_by_noise(name, dummy_calib=True,individual=True,baseline_inactive=bl_values)


    if scan == 3:
         time.sleep(5) 
         print(" threshold scan, all other channels deactive, baseline moved to  15, to scan full range")
         baselines_set =  [15] * 16
         baseline_calib.set_baselines_individual(name,baselines_set)
         baseline_calib.char_noise_by_thresh_scan(name,dummy_calib=True)

    dummy_calib = db.get_calib_json_by_name(name,dummy_calib=True)

    #print(json.dumps(dummy_calib,indent=2, sort_keys=True))
    #threshold scan:
    tsbl_scan_raw = dummy_calib["tsbl_scan_raw"]
    tsbl_range    = dummy_calib["tsbl_range"]
    # baseline scan:
    noise_scan_raw = dummy_calib["noise_scan_raw"]
    noise_range    = dummy_calib["bl_range"]
   
    for i in range(0,16):
     plt.scatter(tsbl_range,tsbl_scan_raw[i],alpha=0.2,label = "{:d}".format(i))

     plt.legend()
     plt.xlabel(" threshold set ( 2 mV / LSB)  ")
     plt.ylabel("mean pulse rate (Hz)")
     #plt.yscale('log') 
    plt.show()

    for i in range(0,16):
     plt.scatter(noise_range,noise_scan_raw[i],alpha=0.2,label = "{:d}".format(i))

     plt.legend()
     plt.xlabel(" baseline setting ( 2 mV / LSB)")
     plt.ylabel("mean pulse rate (Hz)")
     #plt.yscale('log') 
    plt.show()
    fwhm_list = []
    max_list  = []
    rms_list = []
    mean_list = []
#     for i in range(0,16):
#       fwhm_list += [ fwhm(tsbl_range,tsbl_scan_raw[i]) ]
#       rms_list += [ hist_rms(tsbl_range,tsbl_scan_raw[i]) ]
#       mean_list += [ MeanArrays(tsbl_range,tsbl_scan_raw[i]) ]  
#       max_list  += [ np.max(tsbl_scan_raw[i])  ]
#       scaler_list_channel[i][scan_step] = tsbl_scan_raw[i]     
    for i in range(0,16):
      fwhm_list += [ fwhm(noise_range,noise_scan_raw[i]) ]
      rms_list += [ hist_rms(noise_range,noise_scan_raw[i]) ]
      mean_list += [ MeanArrays(noise_range,noise_scan_raw[i])  ]  
      max_list  += [ np.max(noise_scan_raw[i])  ]
      scaler_list_channel[i][scan_step] = noise_scan_raw[i] 
    
    mean_scans[scan][b] = mean_list
    rms_scans[scan][b]  = rms_list    
    fwhm_list = np.array(fwhm_list)
    max_list  = np.array(max_list)
    
    print(name, "average(16chan) noise thresh: mean = ", round(sum(mean_list)/len(mean_list),3), " +- ", round(list_rms(mean_list),3), "  |  RMS  = ", round(sum(rms_list)/len(rms_list),3), " +- ", round(list_rms(rms_list),3) )

    meanAverageTrend_list += [ sum(mean_list)/len(mean_list) ]
    meanAverageTrendError_list += [ list_rms(mean_list) ]
    rmsAverageTrend_list += [ sum(rms_list)/len(rms_list) ]
    rmsAverageTrendError_list += [ list_rms(rms_list) ]    
    



['0014', '0021', '0018', '0022', '0003', '0008', '0027', '0026', '0025', '0004']
0 board = 0014 ------------> peaking time =  20 , gain =  1
1 . scan 
 individual baseline scan, inactive = -15 


TrbException: ('Error while writing trb register.', 41, 'TRB3 Invalid UDP Header, check Firmware')

>ERROR: clearbit of register failed: Termination Status Error
>ERROR: CommonStatusBits: 0x0001, Channel#3 StatusBits: 0x0002
>ERROR:    Ch3_Timeout: timeout error
>ERROR: clearbit of register failed: Termination Status Error
>ERROR: CommonStatusBits: 0x0001, Channel#3 StatusBits: 0x0002
>ERROR:    Ch3_Timeout: timeout error
>ERROR: clearbit of register failed: Termination Status Error
>ERROR: CommonStatusBits: 0x0001, Channel#3 StatusBits: 0x0002
>ERROR:    Ch3_Timeout: timeout error
>ERROR: clearbit of register failed: Termination Status Error
>ERROR: CommonStatusBits: 0x0001, Channel#3 StatusBits: 0x0002
>ERROR:    Ch3_Timeout: timeout error
>WARNING: Status-Bit(s) have been set:
>WARNING: CommonStatusBits: 0x0000, Channel#3 StatusBits: 0x0000
>WARNING:    COM_EndpointReached: no endpoint has been reached
>WARNING: Status-Bit(s) have been set:
>WARNING: CommonStatusBits: 0x0000, Channel#3 StatusBits: 0x0000
>WARNING:    COM_EndpointReached: no endpoint has been reached
>WARNING: Stat

In [ ]:
plt.rcParams["figure.figsize"] = (8,8)
for i in range(len(used_boards)):
#  print(used_boards[i])

 for scan in range(0,nscans): 
    plt.scatter(list(range(0,len(mean_scans[scan][i]))), mean_scans[scan][i],label = "{}, scan {:d}".format(used_boards[i],scan))
    plt.xlabel("channel no")
    plt.ylabel("mean")
#     plt.ylim(ymax = 8, ymin = 0)
 plt.legend()
plt.show()
for i in range(len(used_boards)):
#  print(used_boards[i])
 for scan in range(0,nscans): 
#     plt.scatter(list(range(0,len(mean_scans[scan]))),rms_scans[scan],color='blue',label = "scan {:d}".format(scan))
    plt.scatter(list(range(0,len(mean_scans[scan][i]))),rms_scans[scan][i], label = "{}, scan {:d}".format(used_boards[i],scan))  
    plt.xlabel("channel no")
    plt.ylabel("RMS")
#     plt.ylim(ymax = 6, ymin = 0)
    plt.title("noise width (RMS)")
 plt.legend()
plt.show()
plt.rcParams["figure.figsize"] = (8,4)
for ch in range(0,16):
    print("channel : ",ch)
    for scan in range(0,nscans): 
     plt.plot(noise_range,scaler_list_channel[ch][scan],'-ok',label = "scan {:d},ch {:d}".format(scan,ch))

     plt.legend()
#      plt.xlabel("threshold (at maximum baseline setting)")
     plt.xlabel("baseline ")    
     plt.ylabel("mean pulse rate (Hz)")
     #plt.yscale('log') 
    plt.show()


In [ ]:
#archive(label="baseline_individual_scan_board0021_4xLVDS-Ferrites")